In [1]:
import numpy as np
import pandas as pd
import math

/lib/python3.9/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


#### Compute the associated matrices and coefficients 
Please refer to the document for the detail of the computations. The ReadMe is essential to understand the method. Here is a small summary of comments / techniques used to ease the computations : 
* matrices filling : to fill the upper and lower diagonals of a squared matrix $N \in \mathcal{M}_n(\mathbb{R})$, we decompose it to a squared matrix of size $(n-1,n-1)$, and fill the diagonal : 
    * $N_{bis} = (N_{ij})_{2\leq i \leq n,1\leq j \leq n-1}$ for the lower diagonal 
    * $N_{bis} = (N_{ij})_{1\leq i \leq n-1,2\leq j \leq n}$ for the upper diagonal 
* use a loop to solve the system : strangely, recursive method is not working
* for coefficients except $0$ and $N$, we use whatever number between as the coefficient remains the same
* it remains to do weighting by exponential interpolation for even sizes (odd steps numbers) 

In [2]:
def matrices(r,sigma,T,nb_steps_time,N,theta,delta,X_N):
    
    def coeffs(r,sigma,T,nb_steps_time,N,i,theta,delta):
        alpha,beta,gamma = -0.5*sigma**2,0.5*(sigma**2)-r,r
        dx,dt=10*sigma*math.sqrt(T)/N,T/nb_steps_time
        if i!=0 and i!=N:
            omega = theta*dt*(2*alpha/dx**2 -gamma)-1
            a = (1-theta)*dt*(-2*alpha/dx**2+gamma)-1
            b=theta*dt*(alpha/dx**2-beta/(2*dx))
            c =theta*dt*(alpha/dx**2+beta/(2*dx))
            d=(1-theta)*dt*(alpha/dx**2+beta/(2*dx))
            e=(1-theta)*dt*(alpha/dx**2-beta/(2*dx))    
        if i==0 :
            omega=-theta*dt*(alpha/dx**2-beta/dx+gamma)-1
            a=(1-theta)*dt*(alpha/dx**2-beta/dx+gamma)-1
            b=theta*dt*(beta/dx-2*alpha/dx**2)
            c=(1-theta)*dt*(beta/dx-2*alpha/dx**2)
            d=theta*dt*alpha/dx**2
            e=(1-theta)*dt*alpha/dx**2
        if i==N :
            omega=-theta*dt*(alpha/dx**2+beta/dx+gamma)-1
            a=(1-theta)*dt*(alpha/dx**2+beta/dx+gamma)-1
            b=-theta*dt*(2*alpha/dx**2+beta/dx)
            c=-(1-theta)*dt*(2*alpha/dx**2+beta/dx)
            d=theta*dt*alpha/dx**2
            e=(1-theta)*dt*alpha/dx**2
        return(omega,a,b,c,d,e)
    
    def A_prime(r,sigma,T,nb_steps_time,N,theta,delta):
        A_p = np.zeros((N+1,N+1))

        diag_low_first = [coeffs(r, sigma, T,nb_steps_time, N, 1, theta, delta)[5] for i in range(1,N)]
        diag_low=np.append(diag_low_first,[coeffs(r, sigma, T,nb_steps_time, N, 0, theta, delta)[3]])

        diag_first = [coeffs(r, sigma, T, nb_steps_time,N, 1, theta, delta)[1] for i in range(N)]
        diag_first[0]=coeffs(r, sigma, T,nb_steps_time, N, N, theta, delta)[5]
        diag=np.append(diag_first,[coeffs(r, sigma, T,nb_steps_time, N, 0, theta, delta)[5]])

        diag_up = [coeffs(r, sigma, T,nb_steps_time, N, 5, theta, delta)[4] for i in range(N)]
        diag_up[0]=coeffs(r, sigma, T, nb_steps_time,N, N, theta, delta)[3]

        np.fill_diagonal(A_p[1:,:-1],diag_low)
        np.fill_diagonal(A_p,diag)
        np.fill_diagonal(A_p[:-1,1:],diag_up)

        A_p[0,2]=coeffs(r, sigma, T,nb_steps_time, N, N, theta, delta)[1]
        A_p[N,-3]=coeffs(r, sigma, T,nb_steps_time, N, 0, theta, delta)[1]

        return A_p
    def A_pp(r,sigma,T,nb_steps_time,N,theta,delta):
        A_2p = np.zeros((N+1,N+1))
        diag_low = [coeffs(r, sigma, T, nb_steps_time,N, 1, theta, delta)[2] for i in range(N)] 
        diag_low[-1]=coeffs(r, sigma, T,nb_steps_time, N, 0, theta, delta)[2]

        diag_first=[0 for i in range(N)]
        diag=np.append(diag_first,[coeffs(r, sigma, T,nb_steps_time, N, 0, theta, delta)[4]])
        diag[0]=coeffs(r, sigma, T, nb_steps_time,N, N, theta, delta)[4]

        diag_up = [coeffs(r, sigma, T,nb_steps_time, N, 1, theta, delta)[3] for i in range(N)]
        diag_up[0]=coeffs(r, sigma, T,nb_steps_time, N, N, theta, delta)[2]

        np.fill_diagonal(A_2p[1:,:-1],diag_low)
        np.fill_diagonal(A_2p,diag)
        np.fill_diagonal(A_2p[:-1,1:],diag_up)

        return A_2p
    def Omega(r,sigma,T,nb_steps_time,N,theta,delta):
        L_om= np.full(N+1,coeffs(r, sigma, T, nb_steps_time,N, 1, theta, delta)[0]) 
        Om = np.diag(L_om)
        Om[0,0] = coeffs(r, sigma, T,nb_steps_time, N, N, theta, delta)[0]
        Om[N,N] = coeffs(r, sigma, T,nb_steps_time, N, 0, theta, delta)[0]
        return Om
    
    def system(r,sigma,T,nb_steps_time,N,theta,delta):
        A_p = A_prime(r,sigma,T,nb_steps_time,N,theta,delta)
        A_pprime = A_pp(r,sigma,T,nb_steps_time,N,theta,delta)
        om=Omega(r,sigma,T,nb_steps_time,N,theta,delta)
        A=np.dot(np.linalg.inv(om-A_pprime),A_p)
        b=np.transpose([delta*T/nb_steps_time for i in range(N+1)])
        b=np.dot(np.linalg.inv(om-A_pprime),b)
        return A,b
    
    def solve_system(A,b,X,nb_steps_time):
        for i in range(nb_steps_time):
            X=np.dot(A,X)+b
        return X

    A,b=system(r,sigma,T,nb_steps_time,N,theta,delta)
    return solve_system(A,b,X_N,nb_steps_time)[int((np.shape(solve_system(A,b,X_N,nb_steps_time))[0]-1)/2)] 

#### Define the class ``option`` with option parameters

In [14]:
class option():
    def __init__(self,N,sigma,r,T,nb_steps_time,delta,theta,S0,K,payoff_obj):
        self.strike = K
        self.maturity = T
        self.N=N
        self.vol = sigma
        self.interest_rate = r
        self.steps_time = nb_steps_time
        self.delta = delta
        self.theta=theta
        self.S0 = S0
        self.payoff = payoff_obj.payoff
        
        price = matrices(self.interest_rate,self.vol,self.maturity,self.steps_time,self.N,self.theta,self.delta,self.payoff)
        self.price=price

## Define a payoff class : the one below is a test one (can be used as a modele by the user) for a European Call Option
User shall create a class with an attribute "payoff" , which is a vector of payoff values. Item should then be passed as an argument


In [ ]:
class option_test():
    def __init__(self,S0,sigma,T,N):
        dx = 10*sigma*math.sqrt(T)/N
        Smin=np.log(100)-5*sigma*np.sqrt(T)
        self.payoff = np.array([max(math.exp(Smin+i*dx)-K,0) for i in range(N+1)])

#### We can test our pricing. 
As a reminder, $\theta$ is used to determine the balance between speed and consistency, lies between $0$ and $1$.
<br> As mentioned in the comments in the begining, it remains to interpolate for odd numbers of steps.

In [20]:
N=200
sigma=0.1
S0=100
r=0.02
T=2
K=95
nb_steps_time = 365
delta=0
theta = 0.6

test_payoff = option_test(S0,sigma,T,N)
test_option=option(N,sigma,r,T,nb_steps_time,delta,theta,S0,K,test_payoff)

print("The price of the option is",test_option.price)

The price of the option is 10.83620216352037
